In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Import Lib and Def Func

In [ ]:
import pandas as pd
import time
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, precision_recall_fscore_support, \
    confusion_matrix
import torch
import time
import numpy as np
from sklearn.model_selection import train_test_split ,KFold
import pickle
import re
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier ,GradientBoostingClassifier ,AdaBoostClassifier ,ExtraTreesClassifier ,VotingClassifier ,StackingClassifier
 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
 
import xgboost as xgb
 

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline 

In [ ]:
# read and write model to pickle
def write_to_pickle(Pkl_File_path,model):
  with open(Pkl_File_path, 'wb') as file:  
      pickle.dump(model, file)

def read_pickle_model(path):
  with open(path, 'rb') as file:  
      return pickle.load(file)

In [ ]:
# compute model scores
def compute_metrics(pred,ground_labels):
    labels_all = ground_labels
    preds_all = [round(value) for value in  list(pred)]
    
    precision, recall, f1, _ = precision_recall_fscore_support(labels_all, preds_all ,zero_division=0)
    acc = accuracy_score(labels_all, preds_all)
    confusion_mat = confusion_matrix(labels_all, preds_all)
    # tn, fp, fn, tp = confusiton_mat.ravel()
    out_dict = {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'confusiton_mat': confusion_mat
      }
    return out_dict

In [ ]:
# mapping of the labels to 0,1 
def label_map(x): 
  if x in ['true', 'mostly-true', 'half-true', 'real', 'Real', 0, 'REAL',"TRUE"]:
    return 0
  elif x in ['false', 'pants-fire', 'barely-true', 'fake', 'Fake', 1, 'FAKE',"FALSE"]:
    return 1
  else:return x

**Make sure write to pickele is correctly set**

In [ ]:
def compare_models( X_train, X_val, y_train, y_val,features,key):
  # cla_pred=[]

  # uncommen or add classifiers here

  classifiers = [
            #  ['Logistic Regression',LogisticRegression()],
      #  ['Decision Tree Classification',DecisionTreeClassifier()],
      #  ['Gradient Boosting Classification', GradientBoostingClassifier()],
      #  ['Ada Boosting Classification',AdaBoostClassifier()],
      #  ['Extra Tree Classification', ExtraTreesClassifier(n_estimators=300)],
      #  ['K-Neighbors Classification',KNeighborsClassifier()],
      #  ['Support Vector Classification',SVC()],
       ['Gaussian Naive Bayes',GaussianNB()],
      #  ["xgBoost",xgb.XGBClassifier()]
  ]
  
  for name,model1 in classifiers:
      # print("-----------"+name+"-------------")
      model1.fit(X_train[features],y_train)
      predicted_y = model1.predict(X_val[features])
      d=compute_metrics(predicted_y,y_val)
      # cla_pred.append(d)
      write_to_pickle(outputDirectory+key+"_"+name+".pkl",model1)   #uncomment to write the model to the outputDirectory

      tn, fp, fn, tp = d["confusiton_mat"].ravel() #correct
      print ("{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}".format(d['precision'][0],d['precision'][1], d['recall'][0],d['recall'][1],d['f1'][0],d['f1'][1],d['accuracy'],tn, fp, fn, tp,key,name))

      # dfCm=[key,name,d['precision'][0],d['precision'][1], d['recall'][0],d['recall'][1],d['f1'][0],d['f1'][1],d['accuracy'],tn, fp, fn, tp]
      # results.append(dfCm)

In [ ]:
def normalize(dataFrame,features , parameterDict={}):
  dataframe=dataFrame.copy()
  normDict={}
  for column in dataframe[features].columns.tolist():
    Q1=dataframe[column].quantile(0.25)
    Q3=dataframe[column].quantile(0.75)

    IQR=(Q3-Q1)
    minV=Q1 - 1.5*IQR
    maxV=Q3 + 1.5*IQR

    if (bool(parameterDict)):
      minV = parameterDict[column]["minV"]
      maxV = parameterDict[column]["maxV"]
      print("using parameterDict")


    temp=dataframe[column].copy()
  
    if ( column not in ["qn_symbol_per_sentence" , "num_exclamation_per_sentence" ,"lexical_diversity" ,"url_count_per_sentence"] ) :
      dataframe[column]=dataframe[column].apply(lambda x:minV if x< minV else maxV if x>maxV else x)

      mean = dataframe[column].mean()
      std  = dataframe[column].std() 
      if (bool(parameterDict)):
        mean = parameterDict[column]["mean"]
        std = parameterDict[column]["std"]
        print("using parameterDict")

      try:
        dataframe[column]=dataframe[column].apply(lambda x:  (x-mean)/std )
      except:
        print(column) 

    else:
      dataframe[column]=dataframe[column].apply(lambda x : 1 if x>0 else 0)
      mean = dataframe[column].mean()
      std  = dataframe[column].std() 
      # print("col",column)

    
    summaryDict={"Q1":Q1,"Q3":Q3,"IQR":IQR,"minV":minV,"maxV":maxV,"mean":mean,"std":std}
    normDict[column]=summaryDict
  return dataframe ,normDict

In [ ]:
# !cp /content/model/All_VotingClassifier_hard.pkl "/content/drive/Shareddrives/[FYP] Fake News Detection/Final/Final_Voting_model_5_dataset/"

In [ ]:
def predict_for_model(X_val, y_val,features,dataset_name,model,model_name):
  # print("-----------"+model_name+"-------------")
    predicted_y = model.predict(X_val[features])
    score=compute_metrics(predicted_y,y_val)

    #Prining the evaluation matrix to the console
    d=score
    predictionScore.append(d)
    tn, fp, fn, tp = d["confusiton_mat"].ravel() #correct
    print ("{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:}\t{:}\t{:}\t{:}\t{:}\t\t\t{:}".format(d['precision'][0],d['precision'][1], d['recall'][0],d['recall'][1],d['f1'][0],d['f1'][1],d['accuracy'],tn, fp, fn, tp,dataset_name,model_name))

# Define global varibles

In [ ]:
# set All_features value to be used by the models 
Semantic_features= [
                    # 'url_count', #
                    # 'qn_symbol', #
                    # 'num_chars', #
                    # 'num_words', #
                    # 'num_sentences', #
                    # 'num_exclamation', #
                    'words_per_sentence',  
                    'characters_per_word',
                    'punctuations_per_sentence', 
                    'get_sentiment_polarity',  
                    'lexical_diversity',
                    'content_word_diversity',
                    'redundancy',
                    'noun',
                    'verb',
                    'adj',
                    'adv',
                    "qn_symbol_per_sentence",
                    "num_exclamation_per_sentence",
                    "url_count_per_sentence"
                    ]

LexMod_Features=   [
                    #  'fake_score', 
                    # 'true_score', 
                     'fake_score1', 
                    'true_score1', 
                     'fake_score2', 
                    'true_score2', 
                    # 'common_score'
                    ]

Emotion_features= [
                    #  'highest_eight_label', 
                    'anger', 
                    'anticipation',
                    'disgust', 
                    'fear',
                    'joy', 
                    'sadness', 
                    'surprise', 
                    'trust'
                    ]
Embed_features = ["embd_true","embd_fake"]

All_features=Semantic_features + Emotion_features +LexMod_Features+Embed_features

**VERIFY THE OUTPUT DIRECTORY AND FEATURE INPUT DIRECTORY PATHS BEFORE EXECUTING**

---



In [ ]:
# set output directory for the models generated 
outputDirectory="/content/model/"
!mkdir model

### NEW 

**VERIFY THE PATHS**
(0) lexicon
(1) semantic
(2) emotion 
(3) embedding
(4) dataset

In [ ]:
# input path of the feature set
# -----------------------------

inputPath = {
    "codalab": [
               "/content/drive/Shareddrives/[FYP] Fake News Detection/Results/CodaLab Covid/CodaLab_with_WELFAKE_Lexicon_Scores_Modified.csv",
               "/content/drive/Shareddrives/[FYP] Fake News Detection/Results/CodaLab Covid/CodaLab Covid_sementic.csv",
               "/content/drive/Shareddrives/[FYP] Fake News Detection/Results/CodaLab Covid/Codalab_emotion_scores_modified.csv",
               "/content/drive/Shareddrives/[FYP] Fake News Detection/Results/Final/CodaLab Covid/CodaLab Covid_embedding_new.csv",
               "/content/drive/Shareddrives/[FYP] Fake News Detection/Datasets/CodaLab Covid/Constraint_English_All.csv" ],
    
    "fakenewsnet" :["/content/drive/Shareddrives/[FYP] Fake News Detection/Results/FakeNewsNet/FakeNewsNet_with_WELFAKE_Lexicon_Scores_Modified.csv",
                    "/content/drive/Shareddrives/[FYP] Fake News Detection/Results/FakeNewsNet/FakeNewsNet_sementic.csv",
                    "/content/drive/Shareddrives/[FYP] Fake News Detection/Results/FakeNewsNet/FakeNewsNet_emotion_scores_modified.csv",
                    "/content/drive/Shareddrives/[FYP] Fake News Detection/Results/Final/FakeNewsNet/FakeNewsNet_embedding_new.csv",
                    "/content/drive/Shareddrives/[FYP] Fake News Detection/Datasets/FakeNewsNet/FakeNewsNet_All.csv"],
           
    "isot":   ["/content/drive/Shareddrives/[FYP] Fake News Detection/Results/ISOT/ISOT_with_WELFAKE_Lexicon_Scores_Modified.csv",
            "/content/drive/Shareddrives/[FYP] Fake News Detection/Results/ISOT/ISOT_sementic.csv",
            "/content/drive/Shareddrives/[FYP] Fake News Detection/Results/ISOT/ISOT_emotion_scores_modified.csv",
            "/content/drive/Shareddrives/[FYP] Fake News Detection/Results/Final/ISOT/ISOT_embedding_new.csv",
            "/content/drive/Shareddrives/[FYP] Fake News Detection/Datasets/ISOT/ISOT.csv"],
           
    "kagglerealfake":["/content/drive/Shareddrives/[FYP] Fake News Detection/Results/Kaggle_real_fake/Kaggle_real_fake_with_WELFAKE_Lexicon_Scores_Modified.csv",
                      "/content/drive/Shareddrives/[FYP] Fake News Detection/Results/Kaggle_real_fake/Kaggle_real_fake_sementic.csv",
                      "/content/drive/Shareddrives/[FYP] Fake News Detection/Results/Kaggle_real_fake/Kaggle_real_fake_emotion_scores_modified.csv",
                      "/content/drive/Shareddrives/[FYP] Fake News Detection/Results/Final/Kaggle_real_fake/Kaggle_real_fake_embedding_new.csv",
                      "/content/drive/Shareddrives/[FYP] Fake News Detection/Datasets/Kaggle_real_fake/fake_or_real_news.csv"] ,
    
    "liar":["/content/drive/Shareddrives/[FYP] Fake News Detection/Results/LIAR/Liar_with_WELFAKE_Lexicon_Scores_Modified.csv",
            "/content/drive/Shareddrives/[FYP] Fake News Detection/Results/LIAR/LIAR_sementic.csv",
            "/content/drive/Shareddrives/[FYP] Fake News Detection/Results/LIAR/LIAR_emotion_scores_modified.csv",
            "/content/drive/Shareddrives/[FYP] Fake News Detection/Results/Final/LIAR/LIAR_embedding_new.csv",
            "/content/drive/Shareddrives/[FYP] Fake News Detection/Datasets/LIAR/Liar_all.csv"],
           
    "FA-KES" :["/content/drive/Shareddrives/[FYP] Fake News Detection/Datasets/FA-KES/FA-KES_title_with_WELFAKE_Lexicon_Scores_Modified.csv",
               "/content/drive/Shareddrives/[FYP] Fake News Detection/Results/FA-KES/FA-KES_title_sementic.csv",
               "/content/drive/Shareddrives/[FYP] Fake News Detection/Datasets/FA-KES/FA-KES_title_emotion_scores_modified.csv",
               "/content/drive/Shareddrives/[FYP] Fake News Detection/Results/Final/FA-KES/FA-KES_title_embedding_new.csv",
               "/content/drive/Shareddrives/[FYP] Fake News Detection/Datasets/FA-KES/FA-KES.csv"         
    ],

    # "Politifact_test" : ["/content/drive/Shareddrives/[FYP] Fake News Detection/Datasets/Politifact_test/Politifact_test_with_WELFAKE_Lexicon_Scores_Modified.csv",
    #            "/content/drive/Shareddrives/[FYP] Fake News Detection/Results/Politifact_test/Politifact_test_sementic.csv",
    #            "/content/drive/Shareddrives/[FYP] Fake News Detection/Datasets/Politifact_test/Politifact_testset_emotion_scores_modified.csv",
    #            "/content/drive/Shareddrives/[FYP] Fake News Detection/Results/Final/Politifact_test/Politifact_test_embedding_new.csv",
    #            "/content/drive/Shareddrives/[FYP] Fake News Detection/Datasets/Politifact_test/Politifact_test.csv",        
    # ],
    "COVID19_test" :[
                "/content/drive/Shareddrives/[FYP] Fake News Detection/Datasets/COVID19_test/COVID19_title_test_with_WELFAKE_Lexicon_Scores_Modified.csv",
               "/content/drive/Shareddrives/[FYP] Fake News Detection/Results/COVID19_test/COVID19_test_title_sementic.csv",
               "/content/drive/Shareddrives/[FYP] Fake News Detection/Datasets/COVID19_test/COVID19_test_title_emotion_scores_modified.csv",
               "/content/drive/Shareddrives/[FYP] Fake News Detection/Results/Final/COVID19_test/COVID19_test_title_embedding_new.csv",
               "/content/drive/Shareddrives/[FYP] Fake News Detection/Datasets/COVID19_test/COVID19_test.csv",     
    ]

}

In [ ]:
!rm -fr model
!mkdir model

In [ ]:
# df=pd.read_csv("/content/drive/Shareddrives/[FYP] Fake News Detection/Results/Politifact_test/Politifact_test_sementic.csv")

In [ ]:
# df.describe()

In [ ]:
# df.target.value_counts()

WARNING compare_models funtion will write the model to the resutls folder 
---
---

In [ ]:
# field name of id , label , text_column of respective datasets 
id={"codalab":"id", "fakenewsnet":"id_1", "isot":"id", "kaggle":"id","kagglerealfake":"id", "liar":"ID", "politifact":"claim_id", "welfake":"id","FA-KES":"unit_id","Politifact_test":"id","COVID19_test":"id"}
label={"codalab":"label", "fakenewsnet":"label", "isot":"label", "kaggle":"label","kagglerealfake":"label","liar":"label", "politifact":"cred_label", "welfake":"label","FA-KES":"label","Politifact_test":"target","COVID19_test":"label"}
features=["lexicon","sementic","sentiment","embedding"]
text_column={"codalab":"tweet", "fakenewsnet":"title", "isot":"text", "kaggle":"text","kagglerealfake":"text", "liar":"statement", "politifact":"text", "welfake":"text","FA-KES":"article_content","Politifact_test":"statement","COVID19_test":"text"}

In [ ]:
for key,value in inputPath.items():
  print("----------",key,"--------------")
  df=pd.read_csv(value[-1])
  print(df[label[key]].value_counts())

---------- codalab --------------
real    5600
fake    5100
Name: label, dtype: int64
---------- fakenewsnet --------------
Real    17441
Fake     5755
Name: label, dtype: int64
---------- isot --------------
1    23481
0    21417
Name: label, dtype: int64
---------- kagglerealfake --------------
REAL    3171
FAKE    3164
Name: label, dtype: int64
---------- liar --------------
half-true      2627
false          2507
mostly-true    2454
barely-true    2103
true           2053
pants-fire     1047
Name: label, dtype: int64
---------- FA-KES --------------
0    418
1    371
Name: label, dtype: int64
---------- COVID19_test --------------
0    2061
1    1058
Name: label, dtype: int64


In [ ]:
## load all data set in to a dataframe dictionary (UPDATE :NORMALIZED)

all_df={}

# for each dataset 
for key,value in inputPath.items():
  print("----------",key,"--------------")
  error=False
  ID=id[key]
  TEXT=text_column[key]
  LABEL=label[key]

  # check whether all 4 features are available 
  for v in range(4):
    if value[v]=="":
      error=True
      print(" Error: missing {:} skipping {:}".format(features[v],key))
      break
  if (not error):
    # read each of the 4 features for the dataset 
    dfLexicon  = pd.read_csv(value[0])
    dfSementic = pd.read_csv(value[1])
    dfSentiment = pd.read_csv(value[2])
    dfEmbedding = pd.read_csv(value[3])

    # combine the features using inner join 

    dff=dfSentiment.merge(dfSementic, how='inner', on=ID,suffixes=('_Sentiment', '_Sementic'))
    dff=dff.merge(dfLexicon, how='inner', on=ID,suffixes=('', '_Lexicon'))
    df=dff.merge(dfEmbedding, how='inner', on=ID,suffixes=('', '_Lang'))

    df=df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1)
 
    df["scores"]=df["scores"].apply(lambda x : list(map(np.float64, x.strip('][').replace('"', '').replace("'","").replace(" ","").split(','))))
    # df["fake_score"]=df["scores"].apply(lambda x:x[0])
    # df["true_score"]=df["scores"].apply(lambda x:x[1])

    df["fake_score1"]=df["scores"].apply(lambda x:x[2])
    df["true_score1"]=df["scores"].apply(lambda x:x[3])

    df["fake_score2"]=df["scores"].apply(lambda x:x[4])
    df["true_score2"]=df["scores"].apply(lambda x:x[5])

    df=df.loc[df["lang"]=="en"]    # filter only english text 
    df["label"]=df[LABEL+"_Sementic"]  #set the label coulmn 



    # if (key=="Politifact_test"):
    #   df=df.loc[df["label"].isin(["FALSE","TRUE"])]

    df["label"]=df["label"].apply(label_map)   # converting labels to 0,1 
    df["id"] = df[ID]
    
    # for codalab and liar include split column as well 
    if (key=="codalab" or key=="liar"):  
      df=df[All_features+["label","id","split_Sementic"]]
    else :
      df=df[All_features+["label","id",]]

    print(df["label"].value_counts())


    df=df.loc[df["label"].isin([0,1])]
    df["label"] = df["label"].astype("int64")
      
    #clean 
    print("null rows : ",df.isnull().any(axis=1).sum())
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    print("inf rows : ",df.isnull().any(axis=1).sum())
    df.dropna(inplace=True)

    #uncomment below code to create a randome sample with equal call size 

    # df = df.groupby('label').apply(lambda x: x.sample(n=min(df["label"].value_counts().to_list()))).reset_index(drop = True)

    # all_df[key],pdict=normalize(df,Semantic_features+Emotion_features)   # normalize 
    all_df[key]=df.copy(deep=True)         
    # print(df["label"].value_counts())
all_df.keys()

---------- codalab --------------


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

0    5583
1    5037
Name: label, dtype: int64
null rows :  0
inf rows :  0
---------- fakenewsnet --------------
0    16414
1     5301
Name: label, dtype: int64
null rows :  0
inf rows :  0
---------- isot --------------
1    22724
0    21416
Name: label, dtype: int64
null rows :  0
inf rows :  0
---------- kagglerealfake --------------
0    3171
1    3123
Name: label, dtype: int64
null rows :  0
inf rows :  0
---------- liar --------------
0    7100
1    5609
Name: label, dtype: int64
null rows :  0
inf rows :  0
---------- FA-KES --------------
0    380
1    352
Name: label, dtype: int64
null rows :  0
inf rows :  0
---------- COVID19_test --------------
0    1925
1     946
Name: label, dtype: int64
null rows :  0
inf rows :  0


dict_keys(['codalab', 'fakenewsnet', 'isot', 'kagglerealfake', 'liar', 'FA-KES', 'COVID19_test'])

In [ ]:
dfAll=all_df["isot"].append(all_df["codalab"],ignore_index = True).append(all_df["fakenewsnet"],ignore_index = True).append(all_df["kagglerealfake"],ignore_index = True).append(all_df["liar"],ignore_index = True)
dfAll=dfAll.sample(frac=1 ,random_state=42)
# .append(all_df["welfake"],ignore_index = True).append(all_df["kagglerealfake"],ignore_index = True).append(all_df["isot"],ignore_index = True) 

In [ ]:
# all_df["All"] = dfAll 

In [ ]:
all_df["All"] ,param=normalize(dfAll,Semantic_features+Emotion_features)

for i in ['codalab', 'fakenewsnet', 'isot', 'kagglerealfake', 'liar']:
  all_df.pop(i)

all_df["COVID19_test"],p=normalize(all_df["COVID19_test"],Semantic_features+Emotion_features,param)
all_df["FA-KES"],p=normalize(all_df["FA-KES"],Semantic_features+Emotion_features,param)
all_df.keys()

In [ ]:
param

{'adj': {'IQR': 3.920274651354121,
  'Q1': 3.2225824915030215,
  'Q3': 7.142857142857142,
  'maxV': 13.023269119888324,
  'mean': 5.253355977159745,
  'minV': -2.6578294855281603,
  'std': 3.6516322405515624},
 'adv': {'IQR': 4.266580478345184,
  'Q1': 0.0,
  'Q3': 4.266580478345184,
  'maxV': 10.666451195862962,
  'mean': 2.863691454355184,
  'minV': -6.399870717517777,
  'std': 2.8664873514992695},
 'anger': {'IQR': 6.551,
  'Q1': 3.754,
  'Q3': 10.305,
  'maxV': 20.1315,
  'mean': 7.678782850500116,
  'minV': -6.0725,
  'std': 5.359614312959114},
 'anticipation': {'IQR': 3.431,
  'Q1': 0.692,
  'Q3': 4.123,
  'maxV': 9.2695,
  'mean': 2.780320162759851,
  'minV': -4.4544999999999995,
  'std': 2.547901031179789},
 'characters_per_word': {'IQR': 0.5774664763628214,
  'Q1': 4.687203712100914,
  'Q3': 5.264670188463735,
  'maxV': 6.130869903007968,
  'mean': 5.00141622763443,
  'minV': 3.8210039975566814,
  'std': 0.5114419055117717},
 'content_word_diversity': {'IQR': 24.20306965761511

In [ ]:
# all_df["liar"]["id"] = all_df["liar"]["ID"]
# all_df["fakenewsnet"]["id"] = all_df["fakenewsnet"]["id_1"] 

In [ ]:
# print("Summary\n Class 0- True news \n Class 1 - False news\n")
# print("{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}".format('prec-t','prec-f', 'rec-t','rec-f','f1-t','f1-f','accu','tn', 'fp', 'fn', 'tp',"dataset","model"))  # correct
# for key ,value in all_df.items():
#   pred = np.argmax(value[["embd_fake","embd_true"]].to_numpy(), axis=1)
#   d=compute_metrics(pred,value[["label"]])
#   tn, fp, fn, tp = d["confusiton_mat"].ravel() #correct
#   print ("{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}".format(d['precision'][0],d['precision'][1], d['recall'][0],d['recall'][1],d['f1'][0],d['f1'][1],d['accuracy'],tn, fp, fn, tp,key,""))

In [ ]:
# print("Summary\n Class 0- True news \n Class 1 - False news\n")
# print("{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}".format('prec-t','prec-f', 'rec-t','rec-f','f1-t','f1-f','accu','tn', 'fp', 'fn', 'tp',"dataset","model"))  # correct
# for key ,value in all_df.items():
#   pred = np.argmax(value[["embd_true","embd_fake"]].to_numpy(), axis=1)
#   d=compute_metrics(pred,value[["label"]])
#   tn, fp, fn, tp = d["confusiton_mat"].ravel() #correct
#   print ("{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}".format(d['precision'][0],d['precision'][1], d['recall'][0],d['recall'][1],d['f1'][0],d['f1'][1],d['accuracy'],tn, fp, fn, tp,key,""))

In [ ]:
# np.argmax(value[["embd_fake","embd_true"]].to_numpy(), axis=1)

array([1, 0, 1, ..., 1, 0, 1])

In [ ]:
# all_df1={}

# for key , value in all_df.items():
#   df = all_df[key].copy(deep=True) 
#   all_df1[key]=df.copy(deep=True)

#   keys = [key1 for key1 , value1 in all_df.items() if (key!=key1)]  #and key1!="isot"
#   df_other=[all_df[key1].copy(deep=True)  for key1 , value1 in all_df.items() if (key!=key1)] # and key1!="isot"
#   df_concat = pd.concat(df_other , ignore_index=True ,keys =keys)

#   all_df1["except_"+key],parameterDict =normalize(df_concat,Semantic_features+Emotion_features)   
#   print("normalizing key dataset")
  
#   all_df1[key],_ =normalize(all_df1[key],Semantic_features+Emotion_features,parameterDict)
#   print(key , all_df1["except_"+key].shape)

In [ ]:
keys

# Single Extra tree classifier


In [ ]:
!rm -fr model
!mkdir model

In [ ]:
# Train using all the features 
print("Summary\n Class 0- True news \n Class 1 - False news\n")
print("{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}".format('prec-t','prec-f', 'rec-t','rec-f','f1-t','f1-f','accu','tn', 'fp', 'fn', 'tp',"dataset","model"))  # correct


for key,value in all_df.items(): #for each dataset in all_df
  # print("----------",key,"--------------")
  df = value
  if (key=="codalab" or key=="liar"):
    df_train = df.loc[df["split_Sementic"]=="train"][All_features+['label']]
    df_test = df.loc[df["split_Sementic"]!="train"][All_features+['label']]  

    X_train=df_train[All_features]
    y_train=df_train["label"]
    X_val=df_test[All_features]
    y_val=df_test["label"]
  else:
    X_train, X_val, y_train, y_val = train_test_split(df[All_features], df["label"], test_size=0.3, random_state=142,stratify=df["label"])
  

  compare_models( X_train, X_val, y_train, y_val ,All_features,key) 

Summary
 Class 0- True news 
 Class 1 - False news

prec-t	prec-f	rec-t	rec-f	f1-t	f1-f	accu	tn	fp	fn	tp	dataset	model
0.633	0.781	0.892	0.428	0.740	0.553	0.672	1990	242	1154	864	codalab	Gaussian Naive Bayes
0.770	0.458	0.955	0.117	0.853	0.186	0.751	4705	220	1404	186	fakenewsnet	Gaussian Naive Bayes
0.999	0.998	0.998	0.999	0.999	0.999	0.999	6414	11	5	6812	isot	Gaussian Naive Bayes
0.972	0.987	0.987	0.971	0.980	0.979	0.979	940	12	27	910	kagglerealfake	Gaussian Naive Bayes
0.564	0.569	0.875	0.196	0.686	0.291	0.565	1206	172	933	227	liar	Gaussian Naive Bayes


In [ ]:
# load models 
models={}
baseModelPath= outputDirectory
predictionScore= []

# trained_models_path={
#   "codalab":baseModelPath+"codalab"+"_Extra Tree Classification.pkl",
#   "fakenewsnet": baseModelPath+"fakenewsnet"+"_Extra Tree Classification.pkl",
#   "isot": baseModelPath+"isot"+"_Extra Tree Classification.pkl",
#   "kagglerealfake": baseModelPath+"kagglerealfake"+"_Extra Tree Classification.pkl",
#   "liar":baseModelPath+"liar"+"_Extra Tree Classification.pkl",
# }

trained_models_path={
  "codalab":baseModelPath+"codalab"+"_Gaussian Naive Bayes.pkl",
  "fakenewsnet": baseModelPath+"fakenewsnet"+"_Gaussian Naive Bayes.pkl",
  "isot": baseModelPath+"isot"+"_Gaussian Naive Bayes.pkl",
  "kagglerealfake": baseModelPath+"kagglerealfake"+"_Gaussian Naive Bayes.pkl",
  "liar":baseModelPath+"liar"+"_Gaussian Naive Bayes.pkl",
}
 

print("{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}".format('prec-t','prec-f', 'rec-t','rec-f','f1-t','f1-f','accu','tn', 'fp', 'fn', 'tp',"dataset","model"))  # correct


# need to load the model one at a time to avoid crash 
for key1,value1 in all_df.items():
  df = value1
  X_val=df[All_features]
  y_val=df["label"]
  for key2,value in trained_models_path.items(): 
    Path =value
    value2=read_pickle_model(Path)
    predict_for_model(X_val,y_val,All_features,key1,value2,key2)

prec-t	prec-f	rec-t	rec-f	f1-t	f1-f	accu	tn	fp	fn	tp	dataset	model
0.637	0.801	0.904	0.428	0.747	0.558	0.678	5048	535	2880	2157	codalab			codalab
0.360	0.342	0.304	0.400	0.330	0.369	0.350	1700	3883	3021	2016	codalab			fakenewsnet
0.256	0.472	0.004	0.987	0.008	0.639	0.470	22	5561	64	4973	codalab			isot
0.278	0.442	0.061	0.826	0.099	0.576	0.424	338	5245	876	4161	codalab			kagglerealfake
0.598	0.566	0.637	0.526	0.617	0.546	0.584	3554	2029	2386	2651	codalab			liar
0.792	0.304	0.652	0.470	0.715	0.369	0.607	10694	5720	2807	2494	fakenewsnet			codalab
0.772	0.504	0.961	0.124	0.856	0.198	0.756	15769	645	4646	655	fakenewsnet			fakenewsnet
0.851	0.244	0.003	0.998	0.007	0.393	0.246	57	16357	10	5291	fakenewsnet			isot
0.788	0.246	0.071	0.941	0.130	0.391	0.283	1159	15255	312	4989	fakenewsnet			kagglerealfake
0.812	0.291	0.487	0.651	0.609	0.402	0.527	7995	8419	1850	3451	fakenewsnet			liar
0.575	0.870	0.946	0.340	0.715	0.489	0.634	20265	1151	15002	7722	isot			codalab
0.799	0.936	0.943	0.776	0.865	0.84

In [ ]:
All_features

**Make sure write to pickele is correctly set**

# Voting Classifier

In [ ]:
!rm -fr model
!mkdir model

In [ ]:
# Train using all the features | Voting claasifier 
n_estimators=300
voting="hard"
print("Summary\n Class 0- True news \n Class 1 - False news\n")
print("{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}".format('prec-t','prec-f', 'rec-t','rec-f','f1-t','f1-f','accu','tn', 'fp', 'fn', 'tp',"dataset","model"))  # correct

for key,value in all_df.items():
  # print("----------",key,"--------------")
  df = all_df[key]
  if (key=="codalab" or key=="liar"):
    df_train = df.loc[df["split_Sementic"]=="train"][All_features+['label']]
    df_test = df.loc[df["split_Sementic"]!="train"][All_features+['label']]  

    X_train=df_train[All_features]
    y_train=df_train["label"]
    X_val=df_test[All_features]
    y_val=df_test["label"]
  else:
    X_train, X_val, y_train, y_val = train_test_split(df[All_features], df["label"], test_size=0.3, random_state=142,stratify=df["label"])
  

  # column transforemers 
  sementic_ct = ColumnTransformer([("sementic","passthrough", Semantic_features )])
  lex_ct_embed_ct = ColumnTransformer([("lexicon","passthrough", LexMod_Features +Embed_features)])
  Emotion_ct = ColumnTransformer([("emotion","passthrough", Emotion_features )])
  # embed_ct = ColumnTransformer([("embedding","passthrough", embed_features )])

  
 
  model_sementic_clf = GaussianNB()  #ExtraTreesClassifier(n_estimators=n_estimators)
  model_lex_emd_clf =   KNeighborsClassifier() # ExtraTreesClassifier(n_estimators=n_estimators)
  model_emotion_clf =   GaussianNB() #ExtraTreesClassifier(n_estimators=n_estimators)
 
# DecisionTreeClassifier
# KNeighborsClassifier

# create pipeline 
  sementic_pipeline = Pipeline([
        ('trans', sementic_ct),
        ('clf', model_sementic_clf)
        ]
  )
  lex_embd_pipeline = Pipeline([
    ('trans', lex_ct_embed_ct),
    ('clf', model_lex_emd_clf )
    ]
  )
  emotion_pipeline = Pipeline([
    ('trans', Emotion_ct),
    ('clf', model_emotion_clf)
    ]
  )
  
  
  estimators = [
    ('sementic_estimator', sementic_pipeline),
    ('lexP_embd_estimator', lex_embd_pipeline),
    ("emotion_estimator" ,emotion_pipeline )
  ]

  final_classifier =VotingClassifier(estimators=estimators ,voting=voting) 
  final_classifier.fit(X_train, y_train)
  write_to_pickle(outputDirectory+key+"_"+"VotingClassifier_"+voting+".pkl",final_classifier) 
  predicted_y= final_classifier.predict(X_val)
  d=compute_metrics(predicted_y,y_val)

  tn, fp, fn, tp = d["confusiton_mat"].ravel() #correct
  print ("{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}".format(d['precision'][0],d['precision'][1], d['recall'][0],d['recall'][1],d['f1'][0],d['f1'][1],d['accuracy'],tn, fp, fn, tp,key,"VotingClassifier_"+voting))

Summary
 Class 0- True news 
 Class 1 - False news

prec-t	prec-f	rec-t	rec-f	f1-t	f1-f	accu	tn	fp	fn	tp	dataset	model
0.553	0.634	0.868	0.245	0.676	0.354	0.568	99	15	80	26	FA-KES	VotingClassifier_hard
0.693	0.679	0.971	0.127	0.809	0.214	0.693	561	17	248	36	COVID19_test	VotingClassifier_hard
0.745	0.846	0.915	0.597	0.821	0.700	0.776	14744	1362	5047	7491	All	VotingClassifier_hard


In [ ]:
models={}
baseModelPath= outputDirectory

trained_models_path={
  # "codalab":baseModelPath+"codalab",
  # "fakenewsnet": baseModelPath+"fakenewsnet",
  # "isot": baseModelPath+"isot",
  # "kagglerealfake": baseModelPath+"kagglerealfake",
  # "liar":baseModelPath+"liar",
  # "FA-KES":baseModelPath+"FA-KES",
  # "Politifact_test":baseModelPath+"Politifact_test",
  "All":baseModelPath+"All"
}
 
print("{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}".format('prec-t','prec-f', 'rec-t','rec-f','f1-t','f1-f','accu','tn', 'fp', 'fn', 'tp',"dataset","model"))  # correct

for key,value in all_df.items():
  df = all_df[key]
  X_val=df[All_features]
  y_val=df["label"]
  for key1,value1 in trained_models_path.items():
    Path =value1+"_VotingClassifier_"+voting+ ".pkl" 
    value2=read_pickle_model(Path)
    predicted_y= value2.predict(X_val)
    d=compute_metrics(predicted_y,y_val)

    tn, fp, fn, tp = d["confusiton_mat"].ravel() #correct
    print ("{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}".format(d['precision'][0],d['precision'][1], d['recall'][0],d['recall'][1],d['f1'][0],d['f1'][1],d['accuracy'],tn, fp, fn, tp,key,key1))


prec-t	prec-f	rec-t	rec-f	f1-t	f1-f	accu	tn	fp	fn	tp	dataset	model
0.514	0.467	0.700	0.284	0.592	0.353	0.500	266	114	252	100	FA-KES	All
0.687	0.388	0.799	0.259	0.739	0.311	0.621	1538	387	701	245	COVID19_test	All
0.756	0.868	0.927	0.617	0.833	0.721	0.791	49770	3914	16022	25772	All	All


In [ ]:
models={}
baseModelPath= outputDirectory+"except_"

trained_models_path={
  "codalab":baseModelPath+"codalab",
  "fakenewsnet": baseModelPath+"fakenewsnet",
  "isot": baseModelPath+"isot",
  "kagglerealfake": baseModelPath+"kagglerealfake",
  "liar":baseModelPath+"liar",
  # "FA-KES":baseModelPath+"FA-KES",
  # "Politifact_test":baseModelPath+"Politifact_test",
  # "All":baseModelPath+"All"
}
 
print("{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}".format('prec-t','prec-f', 'rec-t','rec-f','f1-t','f1-f','accu','tn', 'fp', 'fn', 'tp',"dataset","model"))  # correct

for key in ['codalab', 'fakenewsnet', 'isot', 'kagglerealfake', 'liar']:
  df = all_df1[key]

  # if (key=="codalab" or key=="liar"): 
  #   df_test = df.loc[df["split_Sementic"]!="train"][All_features+['label']]  

  #   X_val=df_test[All_features]
  #   y_val=df_test["label"]
  # else:   
  #   X_val=df[All_features]
  #   y_val=df["label"]
  X_val=df[All_features]
  y_val=df["label"]
  
  

  Path =trained_models_path[key]+"_VotingClassifier_"+voting+ ".pkl" 
  value2=read_pickle_model(Path)
  predicted_y= value2.predict(X_val)
  d=compute_metrics(predicted_y,y_val)

  tn, fp, fn, tp = d["confusiton_mat"].ravel() #correct
  print ("{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}".format(d['precision'][0],d['precision'][1], d['recall'][0],d['recall'][1],d['f1'][0],d['f1'][1],d['accuracy'],tn, fp, fn, tp,key,"except_"+key))


prec-t	prec-f	rec-t	rec-f	f1-t	f1-f	accu	tn	fp	fn	tp	dataset	model


KeyError: ignored

In [ ]:
all_df.keys()

dict_keys(['codalab', 'except_codalab', 'fakenewsnet', 'except_fakenewsnet', 'isot', 'except_isot', 'kagglerealfake', 'except_kagglerealfake', 'liar', 'except_liar'])

In [ ]:
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.datasets import load_iris

In [ ]:
value2.estimators_[0]["clf"]

GaussianNB()

In [ ]:
value2.estimators_[2]

Pipeline(steps=[('trans',
                 ColumnTransformer(transformers=[('emotion', 'passthrough',
                                                  ['anger', 'anticipation',
                                                   'disgust', 'fear', 'joy',
                                                   'sadness', 'surprise',
                                                   'trust'])])),
                ('clf', GaussianNB())])

In [ ]:

plt.figure(figsize=(12,8)) 
tree.plot_tree(, filled=True, fontsize=10)
plt.show()

In [ ]:
from sklearn import set_config

set_config(display="diagram") #text
value2.estimators

# Voting Classifier updated

In [ ]:
# Train using all the features | Voting claasifier 
n_estimators=300
voting="hard"
print("Summary\n Class 0- True news \n Class 1 - False news\n")
print("{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}".format('prec-t','prec-f', 'rec-t','rec-f','f1-t','f1-f','accu','tn', 'fp', 'fn', 'tp',"dataset","model"))  # correct

for key,value in all_df.items():
  # print("----------",key,"--------------")
  df = value
  if (key=="codalab" or key=="liar"):
    df_train = df.loc[df["split_Sementic"]=="train"][All_features+['label']]
    df_test = df.loc[df["split_Sementic"]!="train"][All_features+['label']]  

    X_train=df_train[All_features]
    y_train=df_train["label"]
    X_val=df_test[All_features]
    y_val=df_test["label"]
  else:
    X_train, X_val, y_train, y_val = train_test_split(df[All_features], df["label"], test_size=0.3, random_state=142,stratify=df["label"])
  


  estimators=[
      # ('Decision Tree Classification',DecisionTreeClassifier()),
      # ('Gradient Boosting Classification', GradientBoostingClassifier()),
      ('Ada Boosting Classification',AdaBoostClassifier()),
      ('Extra Tree Classification', ExtraTreesClassifier(n_estimators=300)),
      ('Support Vector Classification',SVC(probability=True)),
      ('Gaussian Naive Bayes',GaussianNB()),
      ("xgBoost",xgb.XGBClassifier()),
      # ("extra tree",ExtraTreesClassifier()),
      # ("RandomForestClassifier",RandomForestClassifier())
  ]
  
  # estimators = [
  #   ('sementic_estimator', sementic_pipeline),
  #   ('lexP_embd_estimator', lex_embd_pipeline),
  #   ("emotion_estimator" ,emotion_pipeline )
  # ]

  final_classifier =VotingClassifier(estimators=estimators ,voting=voting) 
  final_classifier.fit(X_train, y_train)
  write_to_pickle(outputDirectory+key+"_"+"VotingClassifier_"+voting+".pkl",final_classifier) 
  predicted_y= final_classifier.predict(X_val)
  d=compute_metrics(predicted_y,y_val)

  tn, fp, fn, tp = d["confusiton_mat"].ravel() #correct
  print ("{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}".format(d['precision'][0],d['precision'][1], d['recall'][0],d['recall'][1],d['f1'][0],d['f1'][1],d['accuracy'],tn, fp, fn, tp,key,"ExtraTree"+voting))

Summary
 Class 0- True news 
 Class 1 - False news

prec-t	prec-f	rec-t	rec-f	f1-t	f1-f	accu	tn	fp	fn	tp	dataset	model
0.816	0.851	0.876	0.781	0.845	0.814	0.831	1955	277	442	1576	codalab	ExtraTreehard
0.771	0.693	0.986	0.095	0.866	0.167	0.769	4858	67	1439	151	fakenewsnet	ExtraTreehard
0.999	1.000	1.000	0.999	0.999	0.999	0.999	6422	3	4	6813	isot	ExtraTreehard
0.988	0.981	0.981	0.988	0.985	0.985	0.985	934	18	11	926	kagglerealfake	ExtraTreehard
0.576	0.572	0.823	0.281	0.678	0.377	0.575	1134	244	834	326	liar	ExtraTreehard


In [ ]:
models={}
baseModelPath= outputDirectory

trained_models_path={
  "codalab":baseModelPath+"codalab",
  "fakenewsnet": baseModelPath+"fakenewsnet",
  "isot": baseModelPath+"isot",
  "kagglerealfake": baseModelPath+"kagglerealfake",
  "liar":baseModelPath+"liar",
}
 
print("{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}".format('prec-t','prec-f', 'rec-t','rec-f','f1-t','f1-f','accu','tn', 'fp', 'fn', 'tp',"dataset","model"))  # correct

for key1,value1 in all_df.items():
  df = value1
  X_val=df[All_features]
  y_val=df["label"]
  for key2,value in trained_models_path.items():
    Path =value+"_VotingClassifier_"+voting+ ".pkl" 
    value2=read_pickle_model(Path)
    predicted_y= value2.predict(X_val)
    d=compute_metrics(predicted_y,y_val)

    tn, fp, fn, tp = d["confusiton_mat"].ravel() #correct
    print ("{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}".format(d['precision'][0],d['precision'][1], d['recall'][0],d['recall'][1],d['f1'][0],d['f1'][1],d['accuracy'],tn, fp, fn, tp,key1,key2))


prec-t	prec-f	rec-t	rec-f	f1-t	f1-f	accu	tn	fp	fn	tp	dataset	model
0.847	0.884	0.903	0.820	0.874	0.851	0.864	5043	540	909	4128	codalab	codalab
0.539	0.841	0.989	0.063	0.698	0.117	0.550	5523	60	4719	318	codalab	fakenewsnet
0.492	0.462	0.252	0.711	0.333	0.560	0.470	1408	4175	1454	3583	codalab	isot
0.507	0.463	0.359	0.612	0.420	0.527	0.479	2005	3578	1953	3084	codalab	kagglerealfake
0.582	0.675	0.864	0.313	0.696	0.428	0.603	4822	761	3458	1579	codalab	liar
0.802	0.267	0.356	0.728	0.493	0.391	0.447	5841	10573	1444	3857	fakenewsnet	codalab
0.776	0.786	0.990	0.116	0.870	0.203	0.777	16246	168	4684	617	fakenewsnet	fakenewsnet
0.760	0.246	0.260	0.746	0.387	0.370	0.379	4264	12150	1344	3957	fakenewsnet	isot
0.757	0.245	0.361	0.642	0.489	0.354	0.430	5926	10488	1900	3401	fakenewsnet	kagglerealfake
0.781	0.319	0.771	0.332	0.776	0.325	0.664	12654	3760	3543	1758	fakenewsnet	liar
0.716	0.620	0.463	0.827	0.563	0.709	0.650	9921	11495	3937	18787	isot	codalab
0.564	0.896	0.963	0.298	0.711	0.447	0.621	20631	7

In [ ]:
# model_sementic_clf1 =  GaussianNB()  
#   model_sementic_clf2 =  SVC()  
#   model_sementic_clf3 =  LogisticRegression(max_iter=1000)  

#   model_lex_emd_clf1  =  GaussianNB()  
#   model_lex_emd_clf2  =  KNeighborsClassifier()  
#   model_lex_emd_clf3  =  xgb.XGBClassifier()  

#   model_emotion_clf1  =  GaussianNB() 
#   model_emotion_clf2  =  SVC()   
#   model_emotion_clf3  =  LogisticRegression(max_iter=1000)  
 


# # create pipeline 
#   sementic_pipeline1 = Pipeline([
#         ('trans', sementic_ct),
#         ('clf', model_sementic_clf1)
#         ]
#   )
#   sementic_pipeline2 = Pipeline([
#         ('trans', sementic_ct),
#         ('clf', model_sementic_clf2)
#         ]
#   )
#   sementic_pipeline3 = Pipeline([
#         ('trans', sementic_ct),
#         ('clf', model_sementic_clf3)
#         ]
#   )

#   lex_embd_pipeline1 = Pipeline([
#     ('trans', lex_ct_embed_ct),
#     ('clf', model_lex_emd_clf1 )
#     ]
#   )
#   lex_embd_pipeline2 = Pipeline([
#     ('trans', lex_ct_embed_ct),
#     ('clf', model_lex_emd_clf2 )
#     ]
#   )
#   lex_embd_pipeline3 = Pipeline([
#     ('trans', lex_ct_embed_ct),
#     ('clf', model_lex_emd_clf3 )
#     ]
#   )

#   emotion_pipeline1 = Pipeline([
#     ('trans', Emotion_ct),
#     ('clf', model_emotion_clf1)
#     ]
#   )
#   emotion_pipeline2 = Pipeline([
#     ('trans', Emotion_ct),
#     ('clf', model_emotion_clf2)
#     ]
#   )
#   emotion_pipeline3 = Pipeline([
#     ('trans', Emotion_ct),
#     ('clf', model_emotion_clf3)
#     ]
#   )
   
  
  
#   estimators = [
#     ('sementic_estimator_GNB', sementic_pipeline1),
#     ('sementic_estimator_SVC', sementic_pipeline2),
#     ('sementic_estimator_LGR', sementic_pipeline3),
#     ('lexP_embd_estimator_GNB', lex_embd_pipeline1),
#     ('lexP_embd_estimator_KNC', lex_embd_pipeline2),
#     ('lexP_embd_estimator_XGB', lex_embd_pipeline3),
#     ("emotion_estimator_GNB" ,emotion_pipeline1),
#     ("emotion_estimator_SVC" ,emotion_pipeline2),
#     ("emotion_estimator_LGR" ,emotion_pipeline3)
#   ]